# Label speaker by click buttons

By LongGang Pang

Try to use the widgets in jupyter notebook to design one software to label the data using clicking on 
buttons.

In [1]:
filename = 'qa.txt'
txtlines = []
def readtxt():
    with open(filename, 'r') as file_to_read:
        lines = file_to_read.readlines()
        for line in lines:
            line = line.replace('\n','')
            a=line.split('|||')
            txtlines.append(a)
    return txtlines
qatxt = readtxt()
# print(qatxt)

In [2]:
from ipywidgets import *
from IPython.display import display
from IPython.html import widgets
from IPython.display import clear_output
import json
import ast

/home/z/anaconda3/lib/python3.6/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [3]:
class ToButtons(object):
    def __init__(self, input_str):
        self.q_str = input_str[0]
        self.a_str = input_str[1]
        self.input_str = input_str[1].split()
        print("question:",self.q_str)
        print("context:",self.a_str)
        self.res = None
        self.buttons = []
        self.pos_labels = []
        item_layout = Layout(height='40px', min_width='40px', max_width='80px')
        items = [Button(layout=item_layout, 
                        description=self.input_str[i],
                        tooltip = str(i),
#                         value = i,
                        button_style='info')
                 for i in range(len(self.input_str))]
        
        box_layout = Layout(
                    border='3px solid black',
                    width='800px',
                    height='',
                    flex_flow='row wrap',
                    display='flex')
        carousel = Box(children=items, layout=box_layout)
        display(carousel)
        
        for item in items:
#             print(item)
            item.on_click(self.on_button_clicked)
        
            
    # function to deal with the checkbox update button       
    def on_button_clicked(self, b):
#         print("b",b)
#         print("description",b.description)
#         print("tooltip:", b.tooltip)
#         print(int(b.tooltip))
#         self.pos_labels.append(b.value)
        self.pos_labels.append(int(b.tooltip))
#         print("answer",self.pos_labels)
        print("istart:",self.pos_labels[0])
        print("iend:",self.pos_labels[-1]+1)

        if len(self.pos_labels) > 2:
            #raise ValueError("only click the start and the end word")
            print("Warning: click more than 2 times, will use the last click \
                  as the end of the label position")
            
    def return_results(self):
        if len(self.pos_labels) == 0:
            self.pos_labels.append(-1)
        return self.input_str, self.q_str, self.a_str, self.pos_labels[0], self.pos_labels[-1]+1


    
class LabelSpeaker(object):
    def __init__(self, talk_list, progress=0, save_to='label_qa.txt'):
        self.progress = progress
        self.saving_path = save_to
        self.talk_list = talk_list
#         self.sentence_buttons = ToButtons(talk_list[self.progress]['context'])
        self.sentence_buttons = ToButtons(talk_list[self.progress])
        self.submit_layout = Layout(height='40px', min_width='40px')
        self.submit = Button(layout=self.submit_layout, 
                        description="submit",
                       button_style='warning')
        self.submit.on_click(self.on_button_submit)
        display(self.submit)        

        
    def save_one_item(self, progress, sentence, q_str, a_str, istart, iend):
        speaker = None
        if istart != -1: answers = sentence[istart:iend]
        answer=''
        for k in answers:
            if not answer=='':
                answer=answer+' '
            answer=answer+k
        res = {'uid':progress, 'question':q_str, 'context':a_str,#'context':sentence,
               'answer':answer,
               'istart':istart, 'iend':iend}
        with open(self.saving_path, 'a') as f:
            f.write(res.__repr__())
            f.write('\n')
        
    def on_button_submit(self, b):
        sentence, q_str, a_str, istart, iend = self.sentence_buttons.return_results()
        print(sentence[istart:iend],istart,iend)
        self.save_one_item(self.progress, sentence, q_str, a_str, istart, iend)
        clear_output()
#         while True:
        self.progress = self.progress + 1
#         print(sentence[istart:iend],istart,iend)
#         print(self.progress)
#             new_sentence = self.talk_list[self.progress]['context']
        new_sentence = self.talk_list[self.progress]
#             existed, res = check_existence(new_sentence, self.progress)
# #             existed = True
#             if existed: 
#                 self.save_one_item(self.progress, new_sentence,
#                                       res['istart'], res['iend'])
#             else:
#                 break
        #### after check exist 
        self.sentence_buttons = ToButtons(new_sentence)
        self.submit = Button(layout=self.submit_layout, 
                        description="submit",
                       button_style='warning')
        self.submit.on_click(self.on_button_submit)
        display(self.submit)

In [4]:
b2 = LabelSpeaker(qatxt,progress=0)#progress为上次标注的最后一行行号

IndexError: list index out of range